In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
print(tf.__version__)

2.18.0


In [3]:
from google.colab import files
uploaded = files.upload()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/creditcard.csv')
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df.shape

(284807, 31)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [ ]:

df_sc = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming df is already loaded
df_sc = df.copy()

# Now apply scaling
df_sc['Time'] = StandardScaler().fit_transform(df_sc['Time'].values.reshape(-1, 1))
df_sc['Amount'] = StandardScaler().fit_transform(df_sc['Amount'].values.reshape(-1, 1))


In [ ]:
df_sc['Time'] = StandardScaler().fit_transform(df_sc['Time'].values.reshape(-1, 1))
df_sc['Amount'] = StandardScaler().fit_transform(df_sc['Amount'].values.reshape(-1, 1))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df_sc, test_size=0.3, random_state = 10)

In [ ]:
X_train = train[train['Class'] == 0]
X_train = X_train.drop(['Class'], axis=1)

X_test = test.drop(['Class'], axis=1)
y_test = test['Class']

In [ ]:

print(f'X_train shape: {X_train.shape};')
print(f'X_test shape: {X_test.shape}; y_test shape: {y_test.shape}')

X_train shape: (199013, 30);
X_test shape: (85443, 30); y_test shape: (85443,)


In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model


In [ ]:

learning_rate = 0.00001
input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim, ))

encoder = Dense(16, activation='elu', activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(8, activation='relu')(encoder)
encoder = Dense(4, activation='relu')(encoder)

decoder = Dense(8, activation='relu')(encoder)
decoder = Dense(16, activation='relu')(decoder)
decoder = Dense(input_dim, activation='elu')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
autoencoder.compile(optimizer='adam',
                    metrics=['accuracy'],
                    loss='mean_squared_error')


EarlyStop = EarlyStopping(monitor='accuracy', patience=5, verbose=1)

autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 30)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,362 (5.32 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

history = autoencoder.fit(X_train, X_train,
          epochs=20,
          batch_size = 64,
          validation_data = (X_test, X_test),
          callbacks = EarlyStop,
          shuffle=True)

Epoch 1/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.3934 - loss: 0.8139 - val_accuracy: 0.5546 - val_loss: 0.6886
Epoch 2/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.5549 - loss: 0.6360 - val_accuracy: 0.5623 - val_loss: 0.6517
Epoch 3/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.5671 - loss: 0.6068 - val_accuracy: 0.5761 - val_loss: 0.6246
Epoch 4/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.5833 - loss: 0.5793 - val_accuracy: 0.5927 - val_loss: 0.6091
Epoch 5/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.5943 - loss: 0.5574 - val_accuracy: 0.5942 - val_loss: 0.5984
Epoch 6/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5975 - loss: 0.5484 - val_accuracy: 0.6004 - val_loss: 0.5932
Epoch 7/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.6023 - loss: 0.5375 - val_accuracy: 0.6096 - val_loss: 0.5846
Epoch 8/20
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.6037 - loss: 0

In [ ]:

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
prediction = autoencoder.predict(X_test)

In [ ]:
mse = np.mean(np.power(X_test - prediction, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse, 'class': y_test})

In [ ]:

error_df.describe()

In [ ]:

bins = 50
plt.figure(figsize=(12, 6))
plt.hist(error_df[error_df['class'] == 0]['reconstruction_error'], bins, alpha=1, density=True, label='Normal')
plt.hist(error_df[error_df['class'] == 1]['reconstruction_error'], bins, alpha=0.6, density=True, label='Fraud', color='crimson')
plt.legend(loc='upper right')
plt.title("Reconstruction error histogram on test sample")
plt.xlabel("Reconstruction error")
plt.ylabel("Percentage of transactions (%)");
plt.show()

In [ ]:

threshold=3

groups = error_df.groupby('class')
fig, ax = plt.subplots(figsize=(12, 6))

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [ ]:
threshold_prediction = [0 if i < threshold else 1 for i in mse]

print(classification_report(y_test, threshold_prediction))
print("Accuracy Score :")
print(accuracy_score(y_test, threshold_prediction))

In [ ]:

y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df['class'], y_pred)

plt.figure(figsize=(7, 5))
sns.heatmap(conf_matrix, xticklabels=['Normal','Fraud'],
            yticklabels=['Normal','Fraud'],
            annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, threshold_prediction)
auc = roc_auc_score(y_test, threshold_prediction)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()